In [1]:
# Importing libraries

# DS libraries
import pandas as pd
from python_calamine.pandas import pandas_monkeypatch
import numpy as np
import os
import getpass
import re # for regular expresion

# GIS libraries
import geopandas as gpd

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Set paths
# Set Paths.
user = getpass.getuser() # Get user.
if user == 'sebas':
    mainpath = "/Users/"+user+"/Dropbox/Documents/Proyecto_UsosDeTierra/"
else:
    mainpath = "/Users/"+user+"/Dropbox/Documents/Proyecto_UsosDeTierra/"


# 2. Create prerequisite files using Python

documentantion in [NASA] https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Generate%20Earthdata%20Prerequisite%20Files

2a. Create `.dodsrc` and `.urs_cookies` files

In [4]:
from subprocess import Popen
import platform
import os
import shutil

homeDir = os.path.expanduser("~") + os.sep

# Create .urs_cookies and .dodsrc files
with open(homeDir + '.urs_cookies', 'w') as file:
    file.write('')
    file.close()
with open(homeDir + '.dodsrc', 'w') as file:
    file.write('HTTP.COOKIEJAR={}.urs_cookies\n'.format(homeDir))
    file.write('HTTP.NETRC={}.netrc'.format(homeDir))
    file.close()

print('Saved .urs_cookies and .dodsrc to:', homeDir)

# Copy dodsrc to working directory in Windows
if platform.system() == "Windows":  
    shutil.copy2(homeDir + '.dodsrc', os.getcwd())
    print('Copied .dodsrc to:', os.getcwd())

Saved .urs_cookies and .dodsrc to: C:\Users\sebas\
Copied .dodsrc to: c:\Users\sebas\Dropbox\Documents\Proyecto_UsosDeTierra\Código\Municipal


Create `.netrc` file using Native Python libraries

In [8]:
from subprocess import Popen
from getpass import getpass
import platform
import os
import shutil

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

homeDir = os.path.expanduser("~") + os.sep

with open(homeDir + '.netrc', 'w') as file:
    file.write('machine {} login {} password {}'.format(urs, getpass(prompt=prompts[0]), getpass(prompt=prompts[1])))
    file.close()

print('Saved .netrc to:', homeDir)

# Set appropriate permissions for Linux/macOS
if platform.system() != "Windows":
    Popen('chmod og-rw ~/.netrc', shell=True)


Saved .netrc to: C:\Users\sebas\


2c. Create `.netrc` file using earthaccess

In [ ]:
# !pip install earthaccess

  Using cached earthaccess-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached multimethod-2.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached pqdm-0.2.0-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached python_cmr-0.13.0-py3-none-any.whl.metadata (10 kB)
  Using cached tinynetrc-1.3.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached bounded_pool_executor-0.0.3-py3-none-any.whl.metadata (2.7 kB)
Using cached earthaccess-0.14.0-py3-none-any.whl (64 kB)
Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
Using cached multimethod-2.0-py3-none-any.whl (9.8 kB)
Using cached pqdm-0.2.0-py2.py3-none-any.whl (6.8 kB)
Using cached python_cmr-0.13.0-py3-none-any.whl (14 kB)
Using cached tinynetrc-1.3.1-py2.py3-none-any.whl (3.9 kB)
Using cached bounded_pool_executor-0.0.3-py3-none-any.whl (3.4 kB)


In [5]:
import earthaccess

auth = earthaccess.login(strategy="interactive", persist=True)

# 3. *Optional*: Retrieve and Store Earthdata Login Token

In [ ]:
import requests
from base64 import b64encode
from getpass import getpass
import os

# Earthdata Login URL for obtaining the token, and creating one if it doesn't exist
url = 'https://urs.earthdata.nasa.gov/api/users/find_or_create_token'

# Earthdata Login credential prompts
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Get credentials from user input
username = getpass(prompt=prompts[0])
password = getpass(prompt=prompts[1])

# Encode credentials using Base64
credentials = b64encode(f"{username}:{password}".encode('utf-8')).decode('utf-8')

# Headers with the Basic Authorization
headers = {
    'Authorization': f'Basic {credentials}'
}

# Make the POST request to get the token
response = requests.post(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON to get the token
    token_info = response.json()
    token = token_info.get("access_token")
    print("Token retrieved successfully")

    # Define the path for the .edl_token file in the home directory
    token_file_path = os.path.join(os.path.expanduser("~"), ".edl_token")

    # Write the token to the .edl_token file
    with open(token_file_path, 'w') as token_file:
        token_file.write(token)

    print(f"Token saved to {token_file_path}")

else:
    print("Failed to retrieve token:", response.text)